In [1]:
# !pip install -U langchain faiss-cpu langchain-huggingface sentence-transformers tqdm pypdf


## Cài đặt & Nhập thư viện

In [2]:
# -*- coding: utf-8 -*-

# Cài đặt các thư viện nếu cần thiết (chạy một lần)
# !pip install -U langchain faiss-cpu langchain-huggingface sentence-transformers tqdm pypdf

import json
import time
import re
import logging
from pathlib import Path # Để xử lý đường dẫn file tốt hơn
from datetime import datetime
from tqdm.notebook import tqdm # Sử dụng tqdm.notebook cho Jupyter

# Các thành phần LangChain
# Cố gắng import FAISS từ các vị trí phổ biến
try:
    # Thử import từ vị trí mới hơn trước (nếu đã cài langchain-faiss)
    from langchain_faiss import FAISS
    logging.info("Đã import FAISS từ langchain_faiss")
except ImportError:
    try:
         # Nếu không được, thử import từ community (phiên bản cũ hơn)
         from langchain_community.vectorstores import FAISS
         logging.info("Đã import FAISS từ langchain_community.vectorstores (có thể đã cũ)")
    except ImportError:
         logging.error("Không thể import FAISS. Hãy cài đặt 'faiss-cpu' hoặc 'faiss-gpu' và 'langchain-faiss' (nếu cần).")
         FAISS = None # Đặt là None nếu không import được

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document # Import cập nhật

# Thiết lập Logging (ghi log) - Chạy một lần để cấu hình
# Xóa các handler cũ nếu chạy lại cell này nhiều lần để tránh log trùng lặp
root_logger = logging.getLogger()
if root_logger.hasHandlers():
    for handler in root_logger.handlers[:]: # Lặp qua bản sao của list
        handler.close()
        root_logger.removeHandler(handler)

logging.basicConfig(level=logging.INFO, # Mức độ log (INFO, WARNING, ERROR)
                    format='%(asctime)s - %(levelname)s - %(message)s', # Định dạng log
                    handlers=[logging.FileHandler("processing_log_vi_jupyter_faiss_final.txt", mode='w', encoding='utf-8'), # Ghi vào file
                              logging.StreamHandler()]) # Hiển thị trên console

logging.info("Đã nhập thư viện (sử dụng FAISS) và cấu hình logging.")

# Kiểm tra lại import FAISS
if FAISS is None:
    logging.error("Import FAISS thất bại. Các ô tiếp theo liên quan đến FAISS sẽ không hoạt động.")

2025-04-21 03:00:15,199 - INFO - Đã nhập thư viện (sử dụng FAISS) và cấu hình logging.


## Cấu hình Các tham số

In [3]:
# --- Cấu hình ---
JSON_FILE_PATH = Path("data_vbpl_boyte_full_details.json") # <<== QUAY LẠI DÙNG FILE PATH
PERSIST_DIRECTORY = Path("db_faiss_phapluat_yte_full_final") # <<== Thư mục lưu index FAISS cho bộ đầy đủ
COLLECTION_NAME = 'vanban_phapluat_yte_full_faiss_final' # Tên logic

MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder" # 
# MODEL_NAME = "keepitreal/vietnamese-sbert" # <<== Hoặc thử model này
# MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
CHUNK_SIZE = 1000 # Số ký tự tối đa mỗi đoạn (chunk)
CHUNK_OVERLAP = 150 # Số ký tự chồng lấn giữa các đoạn
# Các dấu phân tách được tinh chỉnh
SEPARATORS = [
    "\nChương ", "\nMục ", "\nĐiều ",                 # Các yếu tố cấu trúc
    "\n\n", "\n", ". ", "? ", "! ", " ", ""        # Đoạn văn, câu, từ
]
# Biến để kiểm soát việc có tải và thêm vào index cũ không, hay tạo mới hoàn toàn
LOAD_EXISTING_FAISS_INDEX = True # Đặt là False nếu muốn luôn tạo index mới và ghi đè

logging.info("Đã thiết lập các tham số cấu hình.")
print(f"File dữ liệu JSON: {JSON_FILE_PATH}")
print(f"Thư mục lưu Index FAISS: {PERSIST_DIRECTORY}")
print(f"Model Embedding: {MODEL_NAME}")
print(f"Load index FAISS nếu tồn tại: {LOAD_EXISTING_FAISS_INDEX}")

# Tạo thư mục lưu trữ nếu chưa có
PERSIST_DIRECTORY.mkdir(parents=True, exist_ok=True)
faiss_index_path = str(PERSIST_DIRECTORY) # Đường dẫn thư mục cho FAISS

2025-04-21 03:00:15,319 - INFO - Đã thiết lập các tham số cấu hình.


File dữ liệu JSON: data_vbpl_boyte_full_details.json
Thư mục lưu Index FAISS: db_faiss_phapluat_yte_full_final
Model Embedding: bkai-foundation-models/vietnamese-bi-encoder
Load index FAISS nếu tồn tại: True


## Tải Dữ liệu JSON

In [4]:
def load_json_data(file_path: Path) -> list:
    """Tải dữ liệu từ file JSON."""
    if not file_path.exists():
        logging.error(f"Không tìm thấy file: {file_path}")
        return []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if isinstance(data, list):
            logging.info(f"Đã tải thành công {len(data)} văn bản từ {file_path}")
            return data
        else:
            logging.error(f"Lỗi: File {file_path} không chứa một danh sách JSON.")
            return []
    except json.JSONDecodeError as e:
        logging.error(f"Lỗi: File {file_path} không phải là định dạng JSON hợp lệ. {e}")
        return []
    except Exception as e:
        logging.error(f"Lỗi không xác định khi đọc file {file_path}: {e}")
        return []

logging.info("Đã định nghĩa hàm load_json_data.")

2025-04-21 03:00:15,424 - INFO - Đã định nghĩa hàm load_json_data.


## Phân tích Mã Định danh Văn bản

In [5]:
def parse_document_id(ten_van_ban: str, noi_dung: str) -> str:
    """Trích xuất số hiệu/mã định danh chuẩn của văn bản."""
    # Thử phân tích từ TenVanBan trước
    match = re.search(r"(\d+[\w/.-]+(?:TT-BYT|NĐ-CP|QH\d+|Lệnh|QĐ-BYT|BGDĐT|BTC|BLĐTBXH|BTTTT|BTNMT|BTP|BVHTTDL|BYT|BCT|BNN&PTNT|BKH&ĐT|BCA|BQP|BNG|BNV|CP|UBTVQH\d+|VPCP|TTg|CT|NQ-CP|\w+))", ten_van_ban) # Thêm NQ-CP
    if match:
        return match.group(1)

    # Nếu không được, thử phân tích 'Số: ...' từ NoiDung
    match_nd = re.search(r"Số:\s*([\w/.-]+(?:TT-BYT|NĐ-CP|QH\d+|Lệnh|QĐ-BYT|BGDĐT|BTC|BLĐTBXH|BTTTT|BTNMT|BTP|BVHTTDL|BYT|BCT|BNN&PTNT|BKH&ĐT|BCA|BQP|BNG|BNV|CP|UBTVQH\d+|VPCP|TTg|CT|NQ-CP|\w+))", noi_dung, re.IGNORECASE)
    if match_nd:
        return match_nd.group(1)

    # Nếu vẫn không được, dùng phiên bản đã làm sạch của TenVanBan
    cleaned_name = re.sub(r"^(Thông tư|Nghị định|Luật|Quyết định|Công văn|Lệnh|Nghị quyết)\s+", "", ten_van_ban, flags=re.IGNORECASE).strip() # Thêm Nghị quyết
    return cleaned_name if cleaned_name else f"UnknownID_{ten_van_ban[:20]}"

logging.info("Đã định nghĩa hàm parse_document_id.")

2025-04-21 03:00:15,478 - INFO - Đã định nghĩa hàm parse_document_id.


## - Phân tích Ngày Hiệu lực

In [6]:
def parse_effective_date(date_str: str) -> str | None:
    """Phân tích chuỗi ngày dạng DD/MM/YYYY sang<y_bin_46>-MM-DD."""
    if not date_str or not isinstance(date_str, str):
        return None
    try:
        # Xử lý trường hợp ngày/tháng chỉ có 1 chữ số
        parts = date_str.split('/')
        if len(parts) == 3:
             day = parts[0].zfill(2)
             month = parts[1].zfill(2)
             year = parts[2]
             # Thêm kiểm tra năm hợp lệ
             current_year = datetime.now().year
             if not year.isdigit() or int(year) < 1900 or int(year) > current_year + 10:
                 raise ValueError(f"Năm không hợp lệ: {year}")
             if not month.isdigit() or not (1 <= int(month) <= 12):
                 raise ValueError(f"Tháng không hợp lệ: {month}")
             if not day.isdigit() or not (1 <= int(day) <= 31): # Kiểm tra ngày cơ bản
                 raise ValueError(f"Ngày không hợp lệ: {day}")
             # Kiểm tra ngày tháng năm cụ thể
             datetime.strptime(f"{day}/{month}/{year}", "%d/%m/%Y")
             return f"{year}-{month}-{day}"
        else:
             # logging.warning(f"Định dạng ngày không đúng DD/MM/YYYY: {date_str}") # Log nhiều quá
             return date_str # Trả về gốc nếu không đúng định dạng
    except ValueError as e:
        logging.warning(f"Không thể phân tích ngày '{date_str}': {e}")
        return date_str # Trả về gốc nếu lỗi

logging.info("Đã định nghĩa hàm parse_effective_date.")

2025-04-21 03:00:15,503 - INFO - Đã định nghĩa hàm parse_effective_date.


## Làm sạch Nội dung Văn bản

In [7]:
def clean_content(text: str) -> tuple[str, int, int]:
    """Làm sạch nội dung văn bản, trả về nội dung chính và vị trí bắt đầu/kết thúc."""
    start_index = 0
    end_index = len(text)
    lines = text.splitlines()
    if not lines: return "", 0, 0

    # --- Tìm vị trí bắt đầu nội dung chính ---
    found_start = False
    start_content_markers = [
        r"^\s*Chương\s+[IVXLCDM]+", r"^\s*Phần\s+(?:thứ\s+)?\w+", r"^\s*Điều\s+1\b"
    ]
    title_keywords = ["QUY ĐỊNH", "HƯỚNG DẪN", "SỬA ĐỔI", "BỔ SUNG", "BAN HÀNH"]
    title_line_index = -1
    for i, line in enumerate(lines[:min(len(lines), 30)]):
        if re.match(r"^(BỘ|ỦY BAN|CHÍNH PHỦ|VĂN PHÒNG|CỘNG HÒA|Độc lập)", line.strip(), re.IGNORECASE): continue
        if any(keyword in line.upper() for keyword in title_keywords):
            # Kiểm tra dòng trên có phải ngày tháng hoặc số hiệu không
            if i > 0 and (re.search(r"(?:Hà Nội|TP\. Hồ Chí Minh|.*),\s*ngày\s+\d+\s+tháng\s+\d+\s+năm\s+\d+", lines[i-1]) or re.search(r"Số:\s*\d+", lines[i-1])):
                 title_line_index = i
                 break

    citation_end_index = -1
    search_limit = min(len(lines), 60) # Tăng giới hạn tìm căn cứ
    start_search_citation = title_line_index + 1 if title_line_index != -1 else 0

    for i in range(start_search_citation, search_limit):
        line_strip = lines[i].strip()
        # Tìm dòng căn cứ cuối cùng
        if line_strip.lower().startswith("căn cứ"):
            citation_end_index = i
        # Hoặc dòng cuối cùng kết thúc bằng ; trong khối căn cứ
        elif line_strip.endswith(";") and citation_end_index >= start_search_citation:
             citation_end_index = i
        # Dừng nếu gặp dòng không phải căn cứ, không phải đề nghị, và không rỗng (sau khi đã thấy căn cứ)
        elif citation_end_index != -1 and line_strip and not line_strip.lower().startswith("theo đề nghị"):
            break
        # Dừng nếu gặp dòng "theo đề nghị"
        elif line_strip.lower().startswith("theo đề nghị"):
             # Nếu dòng này nằm ngay sau căn cứ cuối cùng, coi nó là hết căn cứ
             if i == citation_end_index + 1:
                 citation_end_index = i
             break # Luôn dừng khi gặp "theo đề nghị"

    content_start_line_index = citation_end_index + 1
    # Bỏ qua các dòng trống hoặc "Bộ trưởng...", "Theo đề nghị..." sau căn cứ
    while content_start_line_index < len(lines):
          line_strip = lines[content_start_line_index].strip()
          if not line_strip: # Dòng trống
              content_start_line_index += 1
              continue
          # Các chức danh/cụm từ phổ biến cần bỏ qua
          if line_strip.lower().startswith(("bộ trưởng", "thủ tướng chính phủ", "theo đề nghị", "chủ tịch")):
              content_start_line_index += 1
              continue
          # Nếu dòng hiện tại là cấu trúc hoặc có nội dung -> dừng bỏ qua
          if any(re.match(p, line_strip, re.IGNORECASE) for p in start_content_markers) or line_strip:
              found_start = True
              break
          # Nếu không phải các trường hợp trên, vẫn tiếp tục bỏ qua (có thể là tên người đề nghị)
          content_start_line_index += 1


    if found_start:
        start_index = sum(len(l) + 1 for l in lines[:content_start_line_index])
    else: # Fallback nếu không tìm thấy điểm bắt đầu rõ ràng
        start_index = 0
        logging.debug(f"Không thể xác định rõ điểm bắt đầu nội dung cho VB: {text[:150]}...")

    # --- Tìm vị trí kết thúc nội dung chính ---
    end_patterns = [
        # Ưu tiên các marker kết thúc rõ ràng
        r"^\s*PHỤ LỤC\s*\d*", r"^\s*DANH MỤC KÈM THEO", r"^\s*Biểu\s*\d+", r"^\s*Mẫu số\s*\d+",
        r"^\s*Nơi nhận:",
        # Chữ ký (KT., Chức danh)
        r"^\s*KT\.?\s*(?:BỘ TRƯỞNG|THỦ TƯỚNG|CHỦ TỊCH|TỔNG GIÁM ĐỐC|GIÁM ĐỐC)",
        r"^\s*(?:THỨ TRƯỞNG|PHÓ THỦ TƯỚNG|PHÓ CHỦ TỊCH|BỘ TRƯỞNG|THỦ TƯỚNG|CHỦ TỊCH|QUYỀN BỘ TRƯỞNG|CHỦ NHIỆM|TỔNG GIÁM ĐỐC|GIÁM ĐỐC)\b",
        # Tên người ký (heuristic)
        # r"^\s*([A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠƯẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪỬỮỰỲÝỴỶỸĐ]+(\s+[A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠƯẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪỬỮỰỲÝỴỶỸĐ]+)*)$",
    ]
    end_line_index = len(lines)
    search_start_rev = max(0, len(lines) - 100)
    if start_index > 0 :
        start_line_approx = len(text[:start_index].splitlines())
        search_start_rev = max(search_start_rev, start_line_approx)

    found_end_marker = False
    # Tìm từ dưới lên
    for i in range(len(lines) - 1, search_start_rev - 1 , -1):
         line_strip = lines[i].strip()
         # Bỏ qua dòng quá dài hoặc quá ngắn không thể là marker
         if len(line_strip) > 100 or len(line_strip) < 3: continue

         # Kiểm tra các mẫu kết thúc
         if any(re.match(p, line_strip, re.IGNORECASE) for p in end_patterns):
              end_line_index = i
              found_end_marker = True
              # logging.debug(f"Tìm thấy marker kết thúc '{line_strip}' tại dòng {i}")
              break # Dừng ngay khi thấy marker mạnh (Phụ lục, Nơi nhận, Ký thay, Chức danh)

    # Nếu tìm thấy marker, lùi lên bỏ qua các dòng trống hoặc tên người ký/chức danh ngay trên đó
    if found_end_marker:
        temp_idx = end_line_index - 1
        while temp_idx >= 0:
            prev_line_strip = lines[temp_idx].strip()
            is_empty = not prev_line_strip
            # Chức danh thường viết hoa, ít từ
            is_title = prev_line_strip.isupper() and len(prev_line_strip.split()) < 7
            # Tên người ký: heuristic đơn giản (viết hoa chữ cái đầu, có khoảng trắng)
            is_signer_name = bool(re.match(r"^[A-ZÀ-Ỹ][a-zà-ỹ]+(?:\s+[A-ZÀ-Ỹ][a-zà-ỹ]+)+$", prev_line_strip))

            if is_empty or is_title or is_signer_name:
                end_line_index = temp_idx # Cập nhật dòng kết thúc thực sự
                temp_idx -= 1
            else:
                # Dừng khi gặp dòng không phải là trống/chức danh/tên
                break
    # else: Nếu không tìm thấy marker nào, end_line_index vẫn là len(lines)

    # Tính vị trí ký tự kết thúc
    if end_line_index < len(lines):
        end_index = sum(len(l) + 1 for l in lines[:end_line_index])
        end_index = max(0, end_index -1) if end_index > 0 else 0 # Trừ newline cuối
    else:
        end_index = len(text)

    # Đảm bảo end >= start và nội dung không quá ngắn
    end_index = max(start_index, end_index)
    main_content = text[start_index:end_index].strip()
    if len(main_content) < 50: # Nếu nội dung chính quá ngắn sau khi cắt -> có thể lỗi -> trả về gần như toàn bộ
         logging.warning(f"Nội dung chính quá ngắn ({len(main_content)} chars) sau khi làm sạch. Kiểm tra lại VB: {text[:150]}...")
         # Có thể chọn không cắt nếu quá ngắn: return text.strip(), 0, len(text)
         pass # Hiện tại vẫn giữ nội dung đã cắt

    # Xóa các dòng chỉ chứa số trang ở cuối/đầu nếu có
    main_content = re.sub(r'\n\s*\d+\s*$', '', main_content).strip()
    main_content = re.sub(r'^\s*\d+\s*\n', '', main_content).strip()

    return main_content, start_index, end_index

# logging.info("Đã định nghĩa hàm clean_content (cải thiện).") # Bỏ bớt log

## Tìm Yếu tố Cấu trúc

In [8]:
def find_structural_elements(text: str) -> list:
    """Tìm tất cả các yếu tố cấu trúc (Chương, Mục, Điều, Khoản, Điểm) và vị trí của chúng."""
    elements = []
    # Pattern được tối ưu hơn để bắt tiêu đề chính xác hơn
    patterns = {
        # Chương/Mục/Điều: Bắt số hiệu (la mã/thường), dấu chấm tùy chọn, sau đó là tiêu đề (không tham lam) đến hết dòng hoặc đến cấu trúc tiếp theo
        'Chương': r"^\s*Chương\s+([IVXLCDM\d]+)\b\.?\s*(.*?)(?=\n\s*(?:Chương|Mục|Điều|\d+\.|[a-zđ]\))|$)",
        'Mục': r"^\s*Mục\s+(\d+|[IVXLCDM]+)\b\.?\s*(.*?)(?=\n\s*(?:Chương|Mục|Điều|\d+\.|[a-zđ]\))|$)",
        'Điều': r"^\s*Điều\s+(\d+)\b\.?\s*(.*?)(?=\n\s*(?:Chương|Mục|Điều|\d+\.|[a-zđ]\))|$)",
        'Khoản': r"^\s*(\d+)\.\s+",
        'Điểm': r"^\s*([a-zđ])\)\s+",
    }

    for type, pattern in patterns.items():
        for match in re.finditer(pattern, text, re.IGNORECASE | re.MULTILINE):
            identifier = match.group(1).strip()
            # Lấy tiêu đề nếu có group 2
            title = match.group(2).strip().replace('\n', ' ') if len(match.groups()) > 1 else ""
            # Chỉ lấy title nếu nó không bắt đầu bằng cấu trúc khác và không quá dài
            if title and len(title) < 250: # Tăng giới hạn độ dài title
                 pass
            else:
                 title = ""

            elements.append({
                'type': type,
                'identifier': identifier,
                'title': title,
                'start': match.start(),
                'end': match.end()
            })

    elements.sort(key=lambda x: x['start'])
    return elements
logging.info("Đã định nghĩa hàm find_structural_elements.") # Bỏ bớt log

2025-04-21 03:00:15,560 - INFO - Đã định nghĩa hàm find_structural_elements.


## Xác định Ngữ cảnh Cấu trúc

In [9]:
def get_contextual_structure(char_index: int, sorted_elements: list) -> dict:
    """Tìm ngữ cảnh cấu trúc (Chương->Mục->Điều->Khoản->Điểm) cho một vị trí ký tự cụ thể."""
    context = {}
    last_element_of_type = {}
    hierarchy = ['Chương', 'Mục', 'Điều', 'Khoản', 'Điểm']

    current_context = {}
    for element in sorted_elements:
        if element['start'] <= char_index:
            current_context[element['type']] = {
                'identifier': element['identifier'],
                'title': element.get('title', '')
            }
            current_level_index = hierarchy.index(element['type'])
            # Xóa ngữ cảnh cấp thấp hơn khi gặp cấp cao hơn
            to_delete = []
            for existing_type in current_context:
                if existing_type != element['type'] and hierarchy.index(existing_type) > current_level_index:
                     to_delete.append(existing_type)
            for key_to_del in to_delete:
                 del current_context[key_to_del]

            last_element_of_type = current_context.copy()
        else:
            break

    location_parts = []
    context_dict = {}
    for struct_type in hierarchy:
         if struct_type in last_element_of_type:
             info = last_element_of_type[struct_type]
             part = f"{struct_type} {info['identifier']}"
             # Chỉ thêm title cho Chương, Mục, Điều
             if struct_type in ['Chương', 'Mục', 'Điều'] and info['title'] and len(info['title']) < 150:
                 part += f": {info['title'][:100]}..." # Rút gọn title nếu quá dài
             location_parts.append(part)
             context_dict[struct_type] = info['identifier']

    context_dict['location_detail'] = " -> ".join(location_parts) if location_parts else "Không có cấu trúc"
    return context_dict

logging.info("Đã định nghĩa hàm get_contextual_structure.") # Bỏ bớt log

2025-04-21 03:00:15,584 - INFO - Đã định nghĩa hàm get_contextual_structure.


## Tiền xử lý Text trước Embedding

In [10]:
import re # Đảm bảo re đã được import
import unicodedata # Thêm import để chuẩn hóa Unicode nếu cần

def preprocess_text_for_embedding(text: str) -> str:
    """Sửa các lỗi định dạng phổ biến trong text trước khi embedding."""
    if not text: return ""

    # ===> CẢI TIẾN XỬ LÝ DẤU CHẤM <====
    # 1. Xử lý chuỗi dấu chấm ASCII liền nhau (3+)
    text = re.sub(r'\.{3,}', ' ', text)
    # 2. Xử lý ký tự dấu ba chấm Unicode (…) liền nhau (1+)
    text = re.sub(r'…+', ' ', text)
    # 3. (MỚI) Xử lý chuỗi dấu chấm ASCII có thể có khoảng trắng xen kẽ (3+ dấu chấm)
    #    Ví dụ: '. . .', '.    .    .', ' . . . '
    #    Pattern này tìm (dấu chấm theo sau bởi 0 hoặc nhiều khoảng trắng) lặp lại 3 lần trở lên
    text = re.sub(r'(\.\s*){3,}', ' ', text)

    # Thay thế chuỗi dài dấu gạch nối bằng khoảng trắng
    text = re.sub(r'-{3,}', ' ', text)

    # Sửa lỗi dính chữ cụ thể (ví dụ)
    text = text.replace("hoặcc)", "hoặc c)")
    text = text.replace("thi hành1.", "thi hành 1.")
    text = text.replace("năm 2025.2.", "năm 2025. 2.")
    # Thêm các trường hợp khác nếu cần dựa trên lỗi thực tế

    # Thêm khoảng trắng sau dấu câu nếu thiếu (heuristic)
    text = re.sub(r'([.;,:?)])([a-zA-Z0-9À-ỹ])', r'\1 \2', text)

    # Thêm khoảng trắng trước số/chữ cái đầu mục nếu dính liền chữ (heuristic)
    text = re.sub(r'([a-zA-ZÀ-ỹ])(\d+\.|[a-zđ]\))', r'\1 \2', text)

    # Chuẩn hóa khoảng trắng (thay thế nhiều khoảng trắng bằng 1, xóa khoảng trắng đầu/cuối)
    text = re.sub(r'\s+', ' ', text).strip()

    # (Tùy chọn) Chuẩn hóa Unicode về dạng NFKC (thường hữu ích)
    text = unicodedata.normalize('NFKC', text)

    # (Tùy chọn) Loại bỏ ký tự lạ (Whitelist - cần kiểm tra kỹ)
    allowed_chars = r'\w\s\d.,;:?!%()-/+ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲÝỴỶỸửữựỳýỵỷỹ'
    text = re.sub(f'[^{allowed_chars}]', ' ', text, flags=re.UNICODE) # Thay ký tự lạ bằng cách
    text = re.sub(r'\s+', ' ', text).strip() # Dọn dẹp khoảng trắng lại

    return text

logging.info("Đã định nghĩa hàm preprocess_text_for_embedding (cải thiện xử lý dấu chấm và gạch nối).")

2025-04-21 03:00:15,607 - INFO - Đã định nghĩa hàm preprocess_text_for_embedding (cải thiện xử lý dấu chấm và gạch nối).


## Khởi tạo Model Embedding, Kho Vector (Kiểm tra FAISS) và Text Splitter

In [11]:
# --- Khởi tạo các thành phần dùng chung ---
logging.info("--- Khởi tạo Model Embedding ---")
embeddings = None
if 'HuggingFaceEmbeddings' in globals() and HuggingFaceEmbeddings:
    try:
        embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME,
                                         model_kwargs={'device': 'cpu'})
        logging.info(f"Model Embedding '{MODEL_NAME}' đã sẵn sàng.")
    except Exception as e:
        logging.error(f"Lỗi khi khởi tạo model embedding: {e}", exc_info=True)
else:
     logging.error("Thư viện HuggingFaceEmbeddings chưa được import/khởi tạo thành công.")

# Khởi tạo Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    add_start_index=True,
    separators=SEPARATORS
)
logging.info(f"Đã khởi tạo Text Splitter với chunk_size={CHUNK_SIZE}, chunk_overlap={CHUNK_OVERLAP}")

# --- Khởi tạo biến vectordb là None ban đầu ---
vectordb = None
faiss_index_exists = False
if FAISS:
    faiss_file = Path(faiss_index_path).joinpath("index.faiss")
    pkl_file = Path(faiss_index_path).joinpath("index.pkl")
    if faiss_file.exists() and pkl_file.exists():
         faiss_index_exists = True
         logging.info(f"Phát hiện index FAISS đã tồn tại tại: {faiss_index_path}")
         if LOAD_EXISTING_FAISS_INDEX and embeddings:
             logging.info("Đang thử tải index FAISS hiện có...")
             try:
                 vectordb = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
                 logging.info(f"Đã tải index FAISS thành công. Số lượng vector hiện tại: {vectordb.index.ntotal}")
             except EOFError as eof:
                  logging.error(f"Lỗi EOFError khi tải index FAISS: {eof}. Sẽ tạo index mới.", exc_info=True)
                  vectordb = None ; faiss_index_exists = False
                  try: faiss_file.unlink(missing_ok=True); pkl_file.unlink(missing_ok=True); logging.info("Đã xóa file index FAISS bị lỗi.")
                  except OSError as del_err: logging.error(f"Không thể xóa file index FAISS bị lỗi: {del_err}")
             except Exception as e:
                 logging.error(f"Lỗi khác khi tải index FAISS: {e}. Sẽ tạo index mới nếu cần.", exc_info=True)
                 vectordb = None ; faiss_index_exists = False
         elif not LOAD_EXISTING_FAISS_INDEX:
             logging.info("Cấu hình không tải index cũ. Index sẽ được tạo mới và ghi đè.")
             faiss_index_exists = False # Đặt là false để ghi đè ở bước sau
             # Xóa file cũ nếu muốn ghi đè hoàn toàn
             try: faiss_file.unlink(missing_ok=True); pkl_file.unlink(missing_ok=True); logging.info("Đã xóa file index FAISS cũ để ghi đè.")
             except OSError as del_err: logging.error(f"Không thể xóa file index FAISS cũ để ghi đè: {del_err}")

         elif not embeddings:
              logging.warning("Model embedding chưa sẵn sàng, không thể tải index FAISS.")
    else:
         logging.info(f"Chưa có index FAISS tại: {faiss_index_path}. Index sẽ được tạo mới.")
else:
    logging.error("Thư viện FAISS chưa được import thành công.")

if not embeddings: logging.error("Không thể khởi tạo Embedding. Dừng xử lý.")
else: logging.info("Embedding đã sẵn sàng.")
if vectordb: logging.info("Vector store FAISS đã được tải.")
elif faiss_index_exists and not LOAD_EXISTING_FAISS_INDEX: logging.info("Vector store FAISS sẽ được tạo mới (ghi đè index cũ).")
elif not faiss_index_exists: logging.info("Vector store FAISS sẽ được tạo mới.")

2025-04-21 03:00:15,635 - INFO - --- Khởi tạo Model Embedding ---
2025-04-21 03:00:30,237 - WARNING - From c:\Users\phamd\AppData\Local\Programs\Python\Python312\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

2025-04-21 03:00:30,640 - INFO - PyTorch version 2.6.0.dev20241206+cu126 available.
2025-04-21 03:00:30,644 - INFO - TensorFlow version 2.18.0 available.
2025-04-21 03:00:31,269 - INFO - Load pretrained SentenceTransformer: bkai-foundation-models/vietnamese-bi-encoder
2025-04-21 03:00:36,617 - INFO - Model Embedding 'bkai-foundation-models/vietnamese-bi-encoder' đã sẵn sàng.
2025-04-21 03:00:36,617 - INFO - Đã khởi tạo Text Splitter với chunk_size=1000, chunk_overlap=150
2025-04-21 03:00:36,621 - INFO - Chưa có index FAISS tại: db_faiss_phapluat_yte_full_final. Index sẽ được tạo mới.
2025-04-21 03:00:36,621 - INFO - Embedding đã sẵn sàng.
2025-04-21 

## Tải Dữ liệu từ File JSON 

In [12]:
# --- Tải Dữ liệu từ File JSON ---
all_documents_data = [] # Khởi tạo rỗng
if JSON_FILE_PATH.exists():
    logging.info(f"--- Bắt đầu tải dữ liệu từ {JSON_FILE_PATH} ---")
    all_documents_data = load_json_data(JSON_FILE_PATH) # Gọi hàm load_json_data
    if not all_documents_data:
        logging.warning("Không tải được dữ liệu hoặc file rỗng. Kiểm tra lại file JSON và đường dẫn.")
    else:
        logging.info(f"Tổng số văn bản sẽ được xử lý: {len(all_documents_data)}")
else:
    logging.error(f"File dữ liệu không tồn tại: {JSON_FILE_PATH}")

2025-04-21 03:00:36,664 - INFO - --- Bắt đầu tải dữ liệu từ data_vbpl_boyte_full_details.json ---
2025-04-21 03:00:36,826 - INFO - Đã tải thành công 614 văn bản từ data_vbpl_boyte_full_details.json
2025-04-21 03:00:36,827 - INFO - Tổng số văn bản sẽ được xử lý: 614


##  Xử lý và Gom Chunks

In [13]:
# --- Vòng lặp Xử lý Chính ---
processed_doc_count = 0
error_doc_count = 0
total_chunks_prepared = 0
all_processed_chunks = [] # Gom tất cả các chunks hợp lệ ở đây

# Chỉ chạy nếu đã tải được dữ liệu và khởi tạo embeddings/FAISS thành công
if all_documents_data and embeddings and FAISS:
    logging.info(f"--- Bắt đầu xử lý {len(all_documents_data)} văn bản để tạo chunks ---")
    for doc_data in tqdm(all_documents_data, desc="Đang xử lý văn bản"):
        doc_id = "Unknown"
        try:
            # --- Trích xuất Thông tin Cơ bản & Metadata ---
            ten_van_ban = doc_data.get("TenVanBan", "")
            noi_dung = doc_data.get("NoiDung", "")
            if not noi_dung:
                logging.debug(f"Văn bản '{ten_van_ban}' không có nội dung. Bỏ qua.")
                error_doc_count += 1
                continue

            doc_id = parse_document_id(ten_van_ban, noi_dung)
            parsed_date = parse_effective_date(doc_data.get("NgayHieuLuc"))

            base_metadata = {
                "document_id": doc_id,
                "document_type": doc_data.get("LoaiVanBan_ThuocTinh", "UnknownType"),
                "effective_date": parsed_date if parsed_date else doc_data.get("NgayHieuLuc", "N/A"),
                "source_link": doc_data.get("DuongLink", "N/A"),
                "domain": doc_data.get("LinhVuc", "N/A")
            }

            # --- Làm sạch Nội dung ---
            main_content, content_start_offset, content_end_offset = clean_content(noi_dung)
            if not main_content:
                logging.debug(f"Không thể trích xuất nội dung chính cho {doc_id}. Bỏ qua.")
                error_doc_count += 1
                continue

            # --- Tìm các Yếu tố Cấu trúc trong Nội dung chính ---
            structural_elements = find_structural_elements(main_content)

            # --- Phân chia thành các Đoạn (Chunks) ---
            temp_docs = text_splitter.create_documents([main_content], metadatas=[base_metadata])

            doc_chunk_count = 0
            # --- Gán Metadata Cấu trúc và Tiền xử lý Nội dung Chunk ---
            for temp_doc in temp_docs:
                chunk_start_in_main = temp_doc.metadata.get("start_index", 0)
                structure_context = get_contextual_structure(chunk_start_in_main, structural_elements)
                final_metadata = base_metadata.copy()
                final_metadata.update(structure_context)
                final_metadata["start_index_in_main"] = chunk_start_in_main
                if "start_index" in final_metadata: del final_metadata["start_index"]

                # ====> ÁP DỤNG TIỀN XỬ LÝ CHO NỘI DUNG CHUNK <====
                raw_page_content = str(temp_doc.page_content)
                preprocessed_content = preprocess_text_for_embedding(raw_page_content)

                if preprocessed_content:
                    final_chunk_doc = Document(page_content=preprocessed_content, metadata=final_metadata)
                    all_processed_chunks.append(final_chunk_doc)
                    doc_chunk_count += 1
                else:
                    logging.debug(f"Bỏ qua chunk rỗng sau tiền xử lý cho {doc_id}")

            if doc_chunk_count > 0:
                 processed_doc_count += 1
                 total_chunks_prepared += doc_chunk_count
            else:
                 # Chỉ log warning nếu văn bản ban đầu có nội dung đáng kể
                 if len(main_content) > 50:
                      logging.warning(f"Không tạo được chunk hợp lệ nào cho {doc_id} (dù đã có main_content).")
                 error_doc_count += 1

        except Exception as e:
            error_doc_count += 1
            logging.error(f"Lỗi không xác định khi xử lý văn bản (ID: {doc_id}): {e}", exc_info=True)


    logging.info("--- Vòng lặp xử lý hoàn tất ---")
    logging.info(f"Số văn bản đã xử lý thành công (để tạo chunk): {processed_doc_count}")
    logging.info(f"Số văn bản bị bỏ qua/lỗi: {error_doc_count}")
    logging.info(f"Tổng số đoạn (chunks) đã chuẩn bị: {total_chunks_prepared}")

elif not all_documents_data:
    logging.error("Dữ liệu văn bản rỗng hoặc không được tải. Không thể xử lý.")
else:
    logging.error("Embedding hoặc thư viện FAISS chưa được khởi tạo/import thành công. Không thể xử lý.")

2025-04-21 03:00:36,858 - INFO - --- Bắt đầu xử lý 614 văn bản để tạo chunks ---


Đang xử lý văn bản:   0%|          | 0/614 [00:00<?, ?it/s]

2025-04-21 03:00:39,175 - WARNING - Nội dung chính quá ngắn (9 chars) sau khi làm sạch. Kiểm tra lại VB: Toàn quốc...
2025-04-21 03:00:44,546 - INFO - --- Vòng lặp xử lý hoàn tất ---
2025-04-21 03:00:44,549 - INFO - Số văn bản đã xử lý thành công (để tạo chunk): 612
2025-04-21 03:00:44,550 - INFO - Số văn bản bị bỏ qua/lỗi: 2
2025-04-21 03:00:44,551 - INFO - Tổng số đoạn (chunks) đã chuẩn bị: 15498


## Kiểm tra và Báo cáo Chunk lỗi Embedding

In [14]:
# --- KIỂM TRA LẠI EMBEDDING TỪNG CHUNK (SAU TIỀN XỬ LÝ) ---
logging.info(f"--- Bắt đầu kiểm tra embedding cho {len(all_processed_chunks)} chunks đã chuẩn bị ---")
problematic_chunks_indices_after_pp = []
good_chunks_count = 0
good_chunks_temp_list = [] # Tạm lưu chunk tốt nếu cần lọc

# Chỉ chạy nếu có chunks và embeddings
if all_processed_chunks and embeddings:
    for i, chunk_doc in enumerate(tqdm(all_processed_chunks, desc="Kiểm tra embedding sau tiền xử lý")):
        try:
            page_content_str = str(chunk_doc.page_content)
            if not page_content_str.strip():
                 # logging.warning(f"Chunk #{i} rỗng sau tiền xử lý, bỏ qua.") # Giảm log
                 continue
            # Thử embed nội dung chunk đã tiền xử lý
            _ = embeddings.embed_documents([page_content_str])
            good_chunks_count += 1
            good_chunks_temp_list.append(chunk_doc) # Thêm vào list tạm thời
        except IndexError as e:
            logging.error(f"Lỗi IndexError khi embed chunk #{i} (SAU TIỀN XỬ LÝ)")
            doc_id_err = chunk_doc.metadata.get('document_id', 'N/A')
            loc_err = chunk_doc.metadata.get('location_detail', 'N/A')
            logging.error(f"  Văn bản: {doc_id_err}")
            logging.error(f"  Vị trí: {loc_err}")
            content_preview_err = page_content_str[:150]
            logging.error(f"  Nội dung đã tiền xử lý (bắt đầu): '{content_preview_err}...'")
            logging.error(f"  Lỗi chi tiết: {e}")
            problematic_chunks_indices_after_pp.append(i)
        except Exception as e_other:
             logging.error(f"Lỗi khác khi embed chunk #{i} (VB: {chunk_doc.metadata.get('document_id', 'N/A')}): {e_other}", exc_info=True)
             problematic_chunks_indices_after_pp.append(i)

    if problematic_chunks_indices_after_pp:
        logging.warning(f"Tìm thấy {len(problematic_chunks_indices_after_pp)} chunks VẪN gây lỗi embedding sau tiền xử lý tại các vị trí (index): {problematic_chunks_indices_after_pp}")
        logging.info(f"Số chunks embed thành công ước tính: {good_chunks_count}")
        # ====> TÙY CHỌN: Lọc bỏ chunk lỗi để chạy tiếp Ô 12 <====
        # Bỏ comment dòng dưới nếu muốn lọc bỏ chunk lỗi và chỉ xử lý chunk tốt
        all_processed_chunks = good_chunks_temp_list
        logging.info(f"Đã cập nhật 'all_processed_chunks' chỉ chứa {len(all_processed_chunks)} chunks hợp lệ cuối cùng.")
    else:
        logging.info("--- Kiểm tra embedding tất cả các chunk sau tiền xử lý thành công ---")
else:
    logging.warning("Không có chunks để kiểm tra embedding hoặc embedding model chưa sẵn sàng.")

2025-04-21 03:00:44,603 - INFO - --- Bắt đầu kiểm tra embedding cho 15498 chunks đã chuẩn bị ---


Kiểm tra embedding sau tiền xử lý:   0%|          | 0/15498 [00:00<?, ?it/s]

2025-04-21 03:47:51,379 - INFO - --- Kiểm tra embedding tất cả các chunk sau tiền xử lý thành công ---


## Tạo hoặc Cập nhật Index FAISS & Lưu trữ

In [15]:


if 'all_processed_chunks' in locals() and all_processed_chunks and embeddings and FAISS:
    logging.info(f"--- Bắt đầu tạo hoặc cập nhật index FAISS từ {len(all_processed_chunks)} chunks đã chuẩn bị ---")
    start_index_time = time.time()
    save_needed = False
    try:
        if vectordb and faiss_index_exists and LOAD_EXISTING_FAISS_INDEX:
             logging.info(f"Đang thêm {len(all_processed_chunks)} chunks mới vào index FAISS hiện có (Tổng cũ: {vectordb.index.ntotal})...")
             if all_processed_chunks:
                 vectordb.add_documents(documents=all_processed_chunks)
                 logging.info(f"Thêm chunks mới thành công. Tổng số vector mới: {vectordb.index.ntotal}")
                 save_needed = True
             else:
                  logging.info("Không có chunks mới để thêm vào index hiện có.")
        else:
             logging.info("Đang tạo index FAISS mới từ đầu...")
             if all_processed_chunks:
                 vectordb = FAISS.from_documents(documents=all_processed_chunks, embedding=embeddings)
                 logging.info(f"Tạo index FAISS mới thành công với {vectordb.index.ntotal} vectors.")
                 save_needed = True
             else:
                  logging.warning("Không có chunks hợp lệ để tạo index FAISS mới.")
                  vectordb = None

        if vectordb and save_needed:
            logging.info(f"--- Lưu trữ index FAISS vào thư mục '{faiss_index_path}' ---")
            start_save = time.time()
            vectordb.save_local(folder_path=faiss_index_path)
            end_save = time.time()
            logging.info(f"Lưu trữ FAISS thành công sau {end_save - start_save:.2f} giây.")
        elif not save_needed:
             logging.info("Không có thay đổi hoặc không tạo mới index, không cần lưu trữ.")
        else: 
             logging.error("Không thể tạo hoặc cập nhật đối tượng vectordb FAISS. Không lưu trữ.")

    except IndexError as idx_err:
         logging.error(f"Lỗi IndexError khi tạo/thêm vào FAISS (vẫn còn chunk lỗi?): {idx_err}", exc_info=True)
         logging.error("Gợi ý: Chạy lại ô kiểm tra embedding (11.5) để xem chi tiết chunk lỗi HOẶC cải thiện hàm preprocess_text_for_embedding (Ô 8.5) HOẶC thử model embedding khác (Ô 2). Xem xét bỏ comment dòng lọc chunk ở Ô 11.5 để chạy tiếp.")
         vectordb = None 
    except Exception as faiss_err:
         logging.error(f"Lỗi khác khi tạo/cập nhật/lưu index FAISS: {faiss_err}", exc_info=True)
         vectordb = None 

    end_index_time = time.time()
    logging.info(f"Hoàn tất quá trình tạo/cập nhật/lưu index FAISS sau {end_index_time - start_index_time:.2f} giây.")

elif not ('all_processed_chunks' in locals() and all_processed_chunks) and ('processed_doc_count' in locals() and processed_doc_count > 0):
     logging.warning("Đã xử lý văn bản nhưng không có chunks hợp lệ nào được chuẩn bị sau khi lọc lỗi embedding (nếu có). Không tạo/lưu index.")
elif not ('embeddings' in locals() and embeddings):
     logging.error("Model embedding không sẵn sàng. Không thể tạo index.")
elif not ('FAISS' in globals() and FAISS):
     logging.error("Thư viện FAISS không sẵn sàng. Không thể tạo index.")
else:
    logging.info("Không có chunks nào được xử lý hoặc không có dữ liệu đầu vào. Không tạo/lưu index FAISS.")

2025-04-21 03:47:51,433 - INFO - --- Bắt đầu tạo hoặc cập nhật index FAISS từ 15498 chunks đã chuẩn bị ---
2025-04-21 03:47:51,438 - INFO - Đang tạo index FAISS mới từ đầu...
2025-04-21 04:23:43,657 - INFO - Loading faiss with AVX2 support.
2025-04-21 04:23:43,682 - INFO - Successfully loaded faiss with AVX2 support.
2025-04-21 04:23:43,698 - INFO - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.
2025-04-21 04:23:44,897 - INFO - Tạo index FAISS mới thành công với 15498 vectors.
2025-04-21 04:23:44,899 - INFO - --- Lưu trữ index FAISS vào thư mục 'db_faiss_phapluat_yte_full_final' ---
2025-04-21 04:23:45,053 - INFO - Lưu trữ FAISS thành công sau 0.15 giây.
2025-04-21 04:23:45,053 - INFO - Hoàn tất quá trình tạo/cập nhật/lưu index FAISS sau 2153.62 giây.


# --- Kiểm tra Cuối cùng ---


In [16]:
# --- Kiểm tra Cuối cùng ---
logging.info("--- Kiểm tra cuối cùng ---")
# Kiểm tra lại sự tồn tại của file index và các thành phần cần thiết
faiss_file_check = Path(faiss_index_path).joinpath("index.faiss")
pkl_file_check = Path(faiss_index_path).joinpath("index.pkl")

if embeddings and FAISS and faiss_file_check.exists() and pkl_file_check.exists():
    try:
        logging.info(f"Đang tải lại index FAISS từ '{faiss_index_path}' để kiểm tra...")
        # Luôn cần embeddings khi load, và cờ allow_dangerous_deserialization
        vectordb_check = FAISS.load_local(
            folder_path=faiss_index_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )
        final_count = vectordb_check.index.ntotal # Lấy số lượng vector
        logging.info(f"Index FAISS tải thành công. Tổng số đoạn (chunks/vectors): {final_count}")

        # Thử truy vấn nếu có vector
        if final_count > 0:
            logging.info("Thực hiện truy vấn thử nghiệm...")
            query = "Điều kiện cấp giấy chứng nhận thực phẩm xuất khẩu"
            start_query_time = time.time()
            # k là số kết quả trả về
            # Thêm tham số fetch_k để lấy nhiều hơn và lọc lại sau (có thể cải thiện chất lượng)
            docs = vectordb_check.similarity_search(query, k=5, fetch_k=20)
            end_query_time = time.time()
            logging.info(f"Truy vấn hoàn tất sau {end_query_time - start_query_time:.3f} giây.")

            if docs:
                logging.info(f"Truy vấn thử '{query}' tìm thấy {len(docs)} kết quả (top 5):")
                for i, doc in enumerate(docs):
                    print(f"\n--- Kết quả {i+1} ---")
                    print(f"Metadata: {doc.metadata}")
                    # In nội dung cẩn thận hơn
                    content_preview = str(doc.page_content)[:300] if hasattr(doc, 'page_content') else "[Nội dung không hợp lệ]"
                    print(f"Nội dung (300 ký tự đầu): {content_preview}...")
            else:
                logging.info(f"Truy vấn thử '{query}' không tìm thấy kết quả nào.")
        else:
            logging.info("Index FAISS không có vector nào để thực hiện truy vấn.")

    except Exception as e:
        logging.error(f"Lỗi trong quá trình kiểm tra cuối cùng với FAISS: {e}", exc_info=True)
else:
    logging.warning(f"Không tìm thấy index FAISS đã lưu ({faiss_index_path}) hoặc lỗi embedding/FAISS. Không thể kiểm tra.")

2025-04-21 04:23:45,086 - INFO - --- Kiểm tra cuối cùng ---
2025-04-21 04:23:45,089 - INFO - Đang tải lại index FAISS từ 'db_faiss_phapluat_yte_full_final' để kiểm tra...
2025-04-21 04:23:45,265 - INFO - Index FAISS tải thành công. Tổng số đoạn (chunks/vectors): 15498
2025-04-21 04:23:45,265 - INFO - Thực hiện truy vấn thử nghiệm...
2025-04-21 04:23:45,322 - INFO - Truy vấn hoàn tất sau 0.057 giây.
2025-04-21 04:23:45,323 - INFO - Truy vấn thử 'Điều kiện cấp giấy chứng nhận thực phẩm xuất khẩu' tìm thấy 5 kết quả (top 5):



--- Kết quả 1 ---
Metadata: {'document_id': '08/2025/TT-BYT', 'document_type': 'Thông tư', 'effective_date': '2025-03-07', 'source_link': 'https://vbpl.vn/boyte/Pages/vbpq-toanvan.aspx?ItemID=176041&Keyword=', 'domain': 'An toàn thực phẩm', 'Điều': '3', 'Khoản': '2', 'Điểm': 'đ', 'location_detail': 'Điều 3: Giấy chứng nhận đối với thực phẩm xuất khẩu... -> Khoản 2 -> Điểm đ', 'start_index_in_main': 1523}
Nội dung (300 ký tự đầu): Số và thời hạn hiệu lực của giấy chứng nhận cơ sở đủ điều kiện an toàn thực phẩm hoặc tương đương đối với giấy chứng nhận liên quan đến cơ sở sản xuất thực phẩm; e) Tên và địa chỉ của tổ chức, cá nhân xuất khẩu; cơ sở sản xuất; g) Căn cứ trên phiếu kiểm nghiệm của lô sản phẩm thực phẩm xuất khẩu, xá...

--- Kết quả 2 ---
Metadata: {'document_id': '08/2025/TT-BYT', 'document_type': 'Thông tư', 'effective_date': '2025-03-07', 'source_link': 'https://vbpl.vn/boyte/Pages/vbpq-toanvan.aspx?ItemID=176041&Keyword=', 'domain': 'An toàn thực phẩm', 'Điều': '4', 'locat

In [17]:
import time # Thêm import time nếu chưa có

# --- Tải Index và Truy xuất ---
logging.info("--- Bắt đầu kiểm tra và truy xuất từ FAISS ---")

# Kiểm tra các thành phần cần thiết và sự tồn tại của file index
faiss_file_check = Path(faiss_index_path).joinpath("index.faiss")
pkl_file_check = Path(faiss_index_path).joinpath("index.pkl")
vectordb_check = None # Khởi tạo là None

if embeddings and FAISS and faiss_file_check.exists() and pkl_file_check.exists():
    try:
        logging.info(f"Đang tải lại index FAISS từ '{faiss_index_path}'...")
        vectordb_check = FAISS.load_local(
            folder_path=faiss_index_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True # Cần thiết cho FAISS
        )
        final_count = vectordb_check.index.ntotal
        logging.info(f"Index FAISS tải thành công. Tổng số đoạn (chunks/vectors): {final_count}")

    except Exception as e:
        logging.error(f"Lỗi khi tải index FAISS để truy xuất: {e}", exc_info=True)
        vectordb_check = None # Đặt lại nếu lỗi
else:
    logging.warning(f"Không tìm thấy index FAISS đã lưu ({faiss_index_path}) hoặc lỗi embedding/FAISS. Không thể truy xuất.")

# --- Thực hiện truy vấn nếu tải index thành công ---
if vectordb_check and vectordb_check.index.ntotal > 0:
    logging.info("\n--- Bắt đầu thực hiện truy vấn ---")

    # === Ví dụ 1: Similarity Search cơ bản ===
    query1 = "Hồ sơ đề nghị cấp giấy chứng nhận thực phẩm xuất khẩu gồm những gì?"
    k1 = 4 # Lấy 4 kết quả
    logging.info(f"\n[Truy vấn 1: similarity_search(k={k1})]")
    print(f"Câu hỏi: {query1}")
    start_time = time.time()
    results1 = vectordb_check.similarity_search(query1, k=k1)
    end_time = time.time()
    logging.info(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")

    print(f"\nKết quả (Top {len(results1)}):")
    for i, doc in enumerate(results1):
        print(f"\n--- Kết quả {i+1} ---")
        print(f"  Metadata: {doc.metadata}")
        print(f"  Nội dung: {doc.page_content[:300]}...") # In 300 ký tự đầu

    # === Ví dụ 2: Similarity Search với Điểm số ===
    query2 = "Thẩm quyền thu hồi giấy chứng nhận?"
    k2 = 3 # Lấy 3 kết quả
    logging.info(f"\n[Truy vấn 2: similarity_search_with_score(k={k2})]")
    print(f"Câu hỏi: {query2}")
    start_time = time.time()
    results2 = vectordb_check.similarity_search_with_score(query2, k=k2)
    end_time = time.time()
    logging.info(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")

    print(f"\nKết quả (Top {len(results2)}):")
    for i, (doc, score) in enumerate(results2):
        print(f"\n--- Kết quả {i+1} ---")
        print(f"  Score: {score:.4f}") # In điểm số (Lưu ý: điểm thấp hơn thường tốt hơn với FAISS/L2)
        print(f"  Metadata: {doc.metadata}")
        print(f"  Nội dung: {doc.page_content[:300]}...")

    # === Ví dụ 3: Max Marginal Relevance Search (MMR) ===
    # Lấy kết quả đa dạng hơn
    query3 = "Nội dung giấy chứng nhận cần có thông tin gì?"
    k3 = 4 # Lấy 4 kết quả cuối cùng
    fetch_k3 = 20 # Lấy 20 kết quả ban đầu để chọn lọc
    logging.info(f"\n[Truy vấn 3: max_marginal_relevance_search(k={k3}, fetch_k={fetch_k3})]")
    print(f"Câu hỏi: {query3}")
    start_time = time.time()
    results3 = vectordb_check.max_marginal_relevance_search(query3, k=k3, fetch_k=fetch_k3)
    end_time = time.time()
    logging.info(f"Thời gian truy vấn: {end_time - start_time:.3f} giây")

    print(f"\nKết quả (Top {len(results3)} - Đã lọc MMR):")
    for i, doc in enumerate(results3):
        print(f"\n--- Kết quả {i+1} ---")
        print(f"  Metadata: {doc.metadata}")
        print(f"  Nội dung: {doc.page_content[:300]}...")

elif vectordb_check and vectordb_check.index.ntotal == 0:
    logging.warning("Index FAISS đã được tải nhưng không chứa vector nào. Không thể truy vấn.")
else:
    logging.warning("Không thể tải Index FAISS hoặc index rỗng. Bỏ qua bước truy vấn.")

2025-04-21 04:23:45,363 - INFO - --- Bắt đầu kiểm tra và truy xuất từ FAISS ---
2025-04-21 04:23:45,379 - INFO - Đang tải lại index FAISS từ 'db_faiss_phapluat_yte_full_final'...
2025-04-21 04:23:45,507 - INFO - Index FAISS tải thành công. Tổng số đoạn (chunks/vectors): 15498
2025-04-21 04:23:45,518 - INFO - 
--- Bắt đầu thực hiện truy vấn ---
2025-04-21 04:23:45,520 - INFO - 
[Truy vấn 1: similarity_search(k=4)]
2025-04-21 04:23:45,563 - INFO - Thời gian truy vấn: 0.043 giây
2025-04-21 04:23:45,563 - INFO - 
[Truy vấn 2: similarity_search_with_score(k=3)]
2025-04-21 04:23:45,603 - INFO - Thời gian truy vấn: 0.041 giây
2025-04-21 04:23:45,605 - INFO - 
[Truy vấn 3: max_marginal_relevance_search(k=4, fetch_k=20)]
2025-04-21 04:23:45,651 - INFO - Thời gian truy vấn: 0.046 giây


Câu hỏi: Hồ sơ đề nghị cấp giấy chứng nhận thực phẩm xuất khẩu gồm những gì?

Kết quả (Top 4):

--- Kết quả 1 ---
  Metadata: {'document_id': '08/2025/TT-BYT', 'document_type': 'Thông tư', 'effective_date': '2025-03-07', 'source_link': 'https://vbpl.vn/boyte/Pages/vbpq-toanvan.aspx?ItemID=176041&Keyword=', 'domain': 'An toàn thực phẩm', 'Điều': '4', 'location_detail': 'Điều 4: Hồ sơ đề nghị cấp giấy chứng nhận...', 'start_index_in_main': 2378}
  Nội dung: Điều 4. Hồ sơ đề nghị cấp giấy chứng nhận Hồ sơ đề nghị cấp giấy chứng nhận cho 01 (mộ t) lô hàng xuất khẩu hoặc một cơ sở sản xuất thực phẩm xuất khẩu gồm: 1. Đơn đề nghị cấp giấy chứng nhận theo mẫu quy định tại Phụ lục ban hành kèm theo Thông tư này; 2. Giấy chứng nhận cơ sở đủ điều kiện an toàn ...

--- Kết quả 2 ---
  Metadata: {'document_id': '47/2014/TT-BYT', 'document_type': 'Thông tư', 'effective_date': '2015-02-15', 'source_link': 'https://vbpl.vn/boyte/Pages/vbpq-toanvan.aspx?ItemID=43946&Keyword=', 'domain': '', 'Chương': 